In [1]:
from utz import *

from bs4 import BeautifulSoup as bs

In [4]:
path = 'FAUQStats2021.xml'

In [7]:
with open(path, 'r') as f:
    xml = bs(f)
xml

<?xml version="1.0" encoding="utf-8"?><!-- This document provides minimal statistics about Fatal Crashes in
 the state of New Jersey as recorded by the New Jersey State Police.
 These statistics are unofficial and subject to change until the
 associated case is completed by the NJSP Fatal Accident
 Investigation Unit.--><html><body><fauqstats>
<rundate>Mon Apr 04 10:00:11 EDT 2022</rundate>
<statsyear>2021</statsyear>
<county ccode="01" cname="Atlantic">
</county>
<county ccode="02" cname="Bergen">
</county>
<county ccode="03" cname="Burlington">
</county>
<county ccode="04" cname="Camden">
</county>
<county ccode="05" cname="Cape May">
</county>
<county ccode="06" cname="Cumberland">
</county>
<county ccode="07" cname="Essex">
</county>
<county ccode="08" cname="Gloucester">
</county>
<county ccode="09" cname="Hudson">
</county>
<county ccode="10" cname="Hunterdon">
</county>
<county ccode="11" cname="Mercer">
</county>
<county ccode="12" cname="Middlesex">
</county>
<county ccode="13

In [80]:
def parse_file(path):
    with open(path, 'r') as f:
        xml = bs(f)
    children = list(xml.children)
    assert len(children) == 3
    html = children[-1]
    assert html.name == 'html'
    fauqstats = html.body.fauqstats
    assert fauqstats.name == 'fauqstats'
    rundate = fauqstats.rundate.text
    year = fauqstats.statsyear.text
    counties = fauqstats.find_all('county', recursive=False)
    crash_counties = [ county for county in counties if county.municipality ]
    print(f'{len(counties)} "county" entries, {len(crash_counties)} containing "municipality"/crash info')
    records = []
    for county in crash_counties:
        municipality = county.municipality
        assert municipality.name == 'municipality'
        children = get_children(municipality)
        accidents = municipality.find_all('accident', recursive=False)
        if len(children) != len(accidents):
            raise ValueError(f'Found {len(children)} municipality children, but {len(accidents)} accidents: {county}. {accidents}')
        for accident in accidents:
            obj = { child.name: child.text for child in get_children(accident) }
            obj = dict(**county.attrs, **municipality.attrs, **accident.attrs, **obj, )
            records.append(obj)
    
    df = DF(records)
    return df

In [81]:
df = parse_file(path)
df

646 "county" entries, 625 containing "municipality"/crash info


,ccode,cname,mcode,mname,accid,date,time,street,location,fatalities,fatal_d,fatal_p,fatal_t,fatal_b,injuries,highway
0,16,Passaic,1608,Paterson City,11485,12/31/2021,1513,N. 4th St,N. 4th St,1,1,0,0,0,0,NaN
1,06,Cumberland,0613,Upper Deerfield Tws,11458,12/30/2021,1623,NaN,County 552 MP 1.4,1,1,0,0,0,2,552
2,15,Ocean,1514,Lakewood Twsp,11470,12/30/2021,0631,NaN,County 623,1,0,1,0,0,NaN,623
3,04,Camden,0436,Winslow Twsp,11457,12/30/2021,0146,NaN,Atlantic City Expressway MP 39.4,1,1,0,0,0,0,446
4,07,Essex,0714,Newark City,11456,12/29/2021,0432,NaN,New Jersey Turnpike MP 105.8,1,0,0,1,0,NaN,95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
626,14,Morris,1421,Montville Twsp,10649,01/03/2021,1537,NaN,State Highway 46 MP 50.5,1,1,0,0,0,0,46
627,08,Gloucester,0805,Franklin Twsp,10628,01/03/2021,0419,NaN,County 555 MP 24.9,1,0,1,0,0,2,555
628,11,Mercer,1113,West Windsor Twp,10627,01/02/2021,0907,NaN,State Highway 1 MP 11.83,1,1,0,0,0,1,1
629,20,Union,2019,Union Twsp,10626,01/01/2021,0906,NaN,Interstate 78 MP 50.8,1,1,0,0,0,0,78


['xml version="1.0" encoding="utf-8"?',
 ' This document provides minimal statistics about Fatal Crashes in\n the state of New Jersey as recorded by the New Jersey State Police.\n These statistics are unofficial and subject to change until the\n associated case is completed by the NJSP Fatal Accident\n Investigation Unit.',
 <html><body><fauqstats>
 <rundate>Mon Apr 04 10:00:11 EDT 2022</rundate>
 <statsyear>2021</statsyear>
 <county ccode="01" cname="Atlantic">
 </county>
 <county ccode="02" cname="Bergen">
 </county>
 <county ccode="03" cname="Burlington">
 </county>
 <county ccode="04" cname="Camden">
 </county>
 <county ccode="05" cname="Cape May">
 </county>
 <county ccode="06" cname="Cumberland">
 </county>
 <county ccode="07" cname="Essex">
 </county>
 <county ccode="08" cname="Gloucester">
 </county>
 <county ccode="09" cname="Hudson">
 </county>
 <county ccode="10" cname="Hunterdon">
 </county>
 <county ccode="11" cname="Mercer">
 </county>
 <county ccode="12" cname="Middlesex

In [59]:
list(xml.children)[0]

In [13]:
list(xml.children)[1]

' This document provides minimal statistics about Fatal Crashes in\n the state of New Jersey as recorded by the New Jersey State Police.\n These statistics are unofficial and subject to change until the\n associated case is completed by the NJSP Fatal Accident\n Investigation Unit.'

In [15]:
html = list(xml.children)[2]
html

<html><body><fauqstats>
<rundate>Mon Apr 04 10:00:11 EDT 2022</rundate>
<statsyear>2021</statsyear>
<county ccode="01" cname="Atlantic">
</county>
<county ccode="02" cname="Bergen">
</county>
<county ccode="03" cname="Burlington">
</county>
<county ccode="04" cname="Camden">
</county>
<county ccode="05" cname="Cape May">
</county>
<county ccode="06" cname="Cumberland">
</county>
<county ccode="07" cname="Essex">
</county>
<county ccode="08" cname="Gloucester">
</county>
<county ccode="09" cname="Hudson">
</county>
<county ccode="10" cname="Hunterdon">
</county>
<county ccode="11" cname="Mercer">
</county>
<county ccode="12" cname="Middlesex">
</county>
<county ccode="13" cname="Monmouth">
</county>
<county ccode="14" cname="Morris">
</county>
<county ccode="15" cname="Ocean">
</county>
<county ccode="16" cname="Passaic">
</county>
<county ccode="17" cname="Salem">
</county>
<county ccode="18" cname="Somerset">
</county>
<county ccode="19" cname="Sussex">
</county>
<county ccode="20" cn

In [18]:
fauqstats = html.body.fauqstats
fauqstats

<fauqstats>
<rundate>Mon Apr 04 10:00:11 EDT 2022</rundate>
<statsyear>2021</statsyear>
<county ccode="01" cname="Atlantic">
</county>
<county ccode="02" cname="Bergen">
</county>
<county ccode="03" cname="Burlington">
</county>
<county ccode="04" cname="Camden">
</county>
<county ccode="05" cname="Cape May">
</county>
<county ccode="06" cname="Cumberland">
</county>
<county ccode="07" cname="Essex">
</county>
<county ccode="08" cname="Gloucester">
</county>
<county ccode="09" cname="Hudson">
</county>
<county ccode="10" cname="Hunterdon">
</county>
<county ccode="11" cname="Mercer">
</county>
<county ccode="12" cname="Middlesex">
</county>
<county ccode="13" cname="Monmouth">
</county>
<county ccode="14" cname="Morris">
</county>
<county ccode="15" cname="Ocean">
</county>
<county ccode="16" cname="Passaic">
</county>
<county ccode="17" cname="Salem">
</county>
<county ccode="18" cname="Somerset">
</county>
<county ccode="19" cname="Sussex">
</county>
<county ccode="20" cname="Union">

In [61]:
counties = [ child for child in get_children(fauqstats) ]
for county in counties:
    assert county.name == 'county', county

AssertionError: <rundate>Mon Apr 04 10:00:11 EDT 2022</rundate>

In [62]:
rundate = fauqstats.rundate.text
rundate

'Mon Apr 04 10:00:11 EDT 2022'

In [63]:
year = fauqstats.statsyear.text
year

'2021'

In [25]:
counties = fauqstats.find_all('county', recursive=False)
len(crashes)

646

In [26]:
crashes

[<county ccode="01" cname="Atlantic">
 </county>,
 <county ccode="02" cname="Bergen">
 </county>,
 <county ccode="03" cname="Burlington">
 </county>,
 <county ccode="04" cname="Camden">
 </county>,
 <county ccode="05" cname="Cape May">
 </county>,
 <county ccode="06" cname="Cumberland">
 </county>,
 <county ccode="07" cname="Essex">
 </county>,
 <county ccode="08" cname="Gloucester">
 </county>,
 <county ccode="09" cname="Hudson">
 </county>,
 <county ccode="10" cname="Hunterdon">
 </county>,
 <county ccode="11" cname="Mercer">
 </county>,
 <county ccode="12" cname="Middlesex">
 </county>,
 <county ccode="13" cname="Monmouth">
 </county>,
 <county ccode="14" cname="Morris">
 </county>,
 <county ccode="15" cname="Ocean">
 </county>,
 <county ccode="16" cname="Passaic">
 </county>,
 <county ccode="17" cname="Salem">
 </county>,
 <county ccode="18" cname="Somerset">
 </county>,
 <county ccode="19" cname="Sussex">
 </county>,
 <county ccode="20" cname="Union">
 </county>,
 <county ccode="2

In [27]:
crash = crashes[0]

In [28]:
crash

<county ccode="01" cname="Atlantic">
</county>

In [47]:
cshs = list(filter(None, [ crash.municipality for crash in crashes ]))
len(cshs)

625

In [48]:
cshs[0]

<municipality mcode="1608" mname="Paterson City">
<accident accid="11485" date="12/31/2021" time="1513">
<street>N. 4th St</street>
<location>N. 4th St</location>
<fatalities>1</fatalities>
<fatal_d>1</fatal_d>
<fatal_p>0</fatal_p>
<fatal_t>0</fatal_t>
<fatal_b>0</fatal_b>
<injuries>0</injuries>
</accident>
</municipality>

In [50]:
muni = cshs[0]
muni

<municipality mcode="1608" mname="Paterson City">
<accident accid="11485" date="12/31/2021" time="1513">
<street>N. 4th St</street>
<location>N. 4th St</location>
<fatalities>1</fatalities>
<fatal_d>1</fatal_d>
<fatal_p>0</fatal_p>
<fatal_t>0</fatal_t>
<fatal_b>0</fatal_b>
<injuries>0</injuries>
</accident>
</municipality>

In [67]:
crash.attrs

{'accid': '11485', 'date': '12/31/2021', 'time': '1513'}

In [66]:
muni.attrs

{'mcode': '1608', 'mname': 'Paterson City'}

In [65]:
muni['mcode'], muni[]

'1608'

In [54]:
def get_children(tag):
    return [ child for child in tag.children if not isinstance(child, str) ]

In [52]:
children = [ child for child in muni.children if not isinstance(child, str) ]
len(children)

1

In [53]:
crash = children[0]
crash

<accident accid="11485" date="12/31/2021" time="1513">
<street>N. 4th St</street>
<location>N. 4th St</location>
<fatalities>1</fatalities>
<fatal_d>1</fatal_d>
<fatal_p>0</fatal_p>
<fatal_t>0</fatal_t>
<fatal_b>0</fatal_b>
<injuries>0</injuries>
</accident>

In [55]:
get_children(crash)

[<street>N. 4th St</street>,
 <location>N. 4th St</location>,
 <fatalities>1</fatalities>,
 <fatal_d>1</fatal_d>,
 <fatal_p>0</fatal_p>,
 <fatal_t>0</fatal_t>,
 <fatal_b>0</fatal_b>,
 <injuries>0</injuries>]

In [58]:
obj = { child.name: child.text for child in get_children(crash) }
obj

{'street': 'N. 4th St',
 'location': 'N. 4th St',
 'fatalities': '1',
 'fatal_d': '1',
 'fatal_p': '0',
 'fatal_t': '0',
 'fatal_b': '0',
 'injuries': '0'}

In [49]:
crash = cshs[0]
crash

<municipality mcode="1608" mname="Paterson City">
<accident accid="11485" date="12/31/2021" time="1513">
<street>N. 4th St</street>
<location>N. 4th St</location>
<fatalities>1</fatalities>
<fatal_d>1</fatal_d>
<fatal_p>0</fatal_p>
<fatal_t>0</fatal_t>
<fatal_b>0</fatal_b>
<injuries>0</injuries>
</accident>
</municipality>

In [ ]:
crash.

In [38]:
len(list(crash.children))

3

In [39]:
children = list(crash.children)

In [40]:
children

['\n',
 <municipality mcode="1608" mname="Paterson City">
 <accident accid="11485" date="12/31/2021" time="1513">
 <street>N. 4th St</street>
 <location>N. 4th St</location>
 <fatalities>1</fatalities>
 <fatal_d>1</fatal_d>
 <fatal_p>0</fatal_p>
 <fatal_t>0</fatal_t>
 <fatal_b>0</fatal_b>
 <injuries>0</injuries>
 </accident>
 </municipality>,
 '\n']

In [45]:
crash.find('> municipality')

In [42]:
children = crash.findChildren()
len(children)

10

In [44]:
children[0]

<municipality mcode="1608" mname="Paterson City">
<accident accid="11485" date="12/31/2021" time="1513">
<street>N. 4th St</street>
<location>N. 4th St</location>
<fatalities>1</fatalities>
<fatal_d>1</fatal_d>
<fatal_p>0</fatal_p>
<fatal_t>0</fatal_t>
<fatal_b>0</fatal_b>
<injuries>0</injuries>
</accident>
</municipality>

In [36]:
crash.get('street')